In [1]:
#Importing our Packages
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as rmse


In [2]:
#Load Libraries and Create engine Connection

import pymysql
import pandas as pd
import numpy as np
import sqlalchemy
import gc
engine =sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
        drivername="mysql+pymysql",
        username='Estimators',  # e.g. "my-database-user"
        password='Estimator2021',  # e.g. "my-database-password"
        host='34.77.88.127',  # e.g. "127.0.0.1"
        port=3306,  # e.g. 3306
        database='Housing_France',  # e.g. "my-database-name"
    ))

conn = engine.connect().execution_options(stream_results=True)

In [3]:
#Load Libraries and Create engine Connection

import pymysql
import pandas as pd
import numpy as np
import sqlalchemy
import gc

def get_data(querystring,chunk):
    engine =sqlalchemy.create_engine(
    sqlalchemy.engine.url.URL.create(
    drivername="mysql+pymysql",
    username='Estimators',  # e.g. "my-database-user"
    password='Estimator2021',  # e.g. "my-database-password"
    host='34.77.88.127',  # e.g. "127.0.0.1"
    port=3306,  # e.g. 3306
    database='Housing_France',  # e.g. "my-database-name"
    ))
    conn = engine.connect().execution_options(stream_results=True)
    frame = pd.DataFrame()
    for chunk_dataframe in pd.read_sql(
            querystring, conn, chunksize=chunk):
            print(f"Got dataframe w/{len(chunk_dataframe)} rows")
            frame= frame.append(chunk_dataframe)
            # ... do something with dataframe ...
            
    return frame

In [4]:
#Template for getting some data
df = get_data("""select * from data_working_update LIMIT 10000""", 1000)

Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows


In [5]:
data = df.copy()
data.columns

Index(['id_mutation', 'date_mutation', 'nature_mutation', 'valeur_fonciere',
       'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie',
       'code_postal', 'code_commune', 'nom_commune', 'code_departement',
       'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle',
       'ancien_id_parcelle', 'lot1_numero', 'lot1_surface_carrez',
       'lot2_numero', 'lot2_surface_carrez', 'lot3_numero',
       'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez',
       'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local',
       'type_local', 'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [6]:
data.shape

(10000, 37)

# Encoding

## type_local

In [7]:
data[['type_local']].isnull().sum()

type_local    0
dtype: int64

In [8]:
data['type_local'].value_counts()

Maison         6747
Appartement    3252
Dépendance        1
Name: type_local, dtype: int64

In [9]:
data['type_local'].unique()

array(['Appartement', 'Maison', 'Dépendance'], dtype=object)

In [10]:
data['code_type_local'].unique()

array([2, 1, 3])

In [11]:
from sklearn.preprocessing import OneHotEncoder

print(data['type_local'].unique())  # Check unique values for streets (3)

ohe = OneHotEncoder(sparse = False) # Instanciate encoder

ohe.fit(data[['type_local']]) # Fit encoder  ---> OneHotEncoder(sparse=False)

['Appartement' 'Maison' 'Dépendance']


OneHotEncoder(sparse=False)

In [12]:
ohe.categories_

[array(['Appartement', 'Dépendance', 'Maison'], dtype=object)]

In [14]:
type_local_encoded = ohe.transform(data[['type_local']]) # Encode

data["Appartement"], data['Maison'] , data['Dépendance']= \
\
type_local_encoded.T # Transpose encoded type_local 
                                                                            #back into dataframe

In [15]:
data = data.drop(columns= 'type_local')
data.shape

(10000, 39)

In [16]:
data.head(2)

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,...,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,Appartement,Maison,Dépendance
0,2016-1,2016-01-08,Vente,40000,,RUE TONY REVILLON,0560,1750,1370,Saint-Laurent-sur-Saône,...,,,,,0,4.840606,46.304486,1.0,0.0,0.0
1,2016-4,2016-01-08,Vente,90000,,PELUS,B203,1560,1364,Saint-Jean-sur-Reyssouze,...,S,sols,,,1220,5.059020,46.386697,0.0,0.0,1.0


## code_nature_culture

In [17]:
data[['code_nature_culture']].isnull().sum()

code_nature_culture    0
dtype: int64

In [18]:
data['code_nature_culture'].unique()

array(['', 'S', 'AG', 'P', 'T', 'J', 'AB', 'BS', 'CH', 'BT', 'L', 'BM',
       'PA', 'VE', 'BR'], dtype=object)

In [19]:
data['code_nature_culture'].replace("", "NoCodeNature", inplace=True) #Replace NaN by "NoCodeNature"

In [20]:
print(data['code_nature_culture'].unique())  # Check unique values 

ohe = OneHotEncoder(sparse = False) # Instanciate encoder

ohe.fit(data[['code_nature_culture']]) # Fit encoder  ---> OneHotEncoder(sparse=False)

['NoCodeNature' 'S' 'AG' 'P' 'T' 'J' 'AB' 'BS' 'CH' 'BT' 'L' 'BM' 'PA'
 'VE' 'BR']


OneHotEncoder(sparse=False)

In [21]:
ohe.categories_

[array(['AB', 'AG', 'BM', 'BR', 'BS', 'BT', 'CH', 'J', 'L', 'NoCodeNature',
        'P', 'PA', 'S', 'T', 'VE'], dtype=object)]

In [22]:
code_nature_culture_encoded = ohe.transform(data[['code_nature_culture']]) # Encode

In [23]:
code_nature_culture_encoded.T.shape

(15, 10000)

In [24]:
dicts_nature = {}
keys = list(ohe.categories_[0])
values = code_nature_culture_encoded.T.astype(int)
for i,j in enumerate(keys):
    dicts_nature[j] = values[i,:]

result = pd.DataFrame.from_dict(dicts_nature)

print(result.shape)
result.head()

(10000, 15)


,AB,AG,BM,BR,BS,BT,CH,J,L,NoCodeNature,P,PA,S,T,VE
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [25]:
data = data.reset_index(drop=True)

In [26]:
#Concat data_v and result dataframe
data_v1 = pd.concat([data, result], axis = 1)
data_v1.shape

(10000, 54)

In [27]:
data_v1 = data_v1.drop(columns= ['NoCodeNature', 'code_nature_culture'])
data_v1.shape

(10000, 52)

In [28]:
data_v1.head(2)

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,...,BS,BT,CH,J,L,P,PA,S,T,VE
0,2016-1,2016-01-08,Vente,40000,,RUE TONY REVILLON,0560,1750,1370,Saint-Laurent-sur-Saône,...,0,0,0,0,0,0,0,0,0,0
1,2016-4,2016-01-08,Vente,90000,,PELUS,B203,1560,1364,Saint-Jean-sur-Reyssouze,...,0,0,0,0,0,0,0,1,0,0


## code_commune

In [29]:
data_v1[['code_commune']].isnull().sum()

code_commune    0
dtype: int64

In [30]:
print(data_v1['code_commune'].unique())  # Check unique values 

ohe = OneHotEncoder(sparse = False) # Instanciate encoder

ohe.fit(data_v1[['code_commune']]) # Fit encoder  ---> OneHotEncoder(sparse=False)

[1370 1364 1050 1024 1130 1163 1241 1273 1291 1159 1301 1266 1053 1374
 1196 1320 1447 1072 1211 1305 1344 1350 1197 1426 1422 1321 1264 1451
 1195 1232 1230 1016 1026 1179 1203 1304 1306 1346 1127 1229 1025 1042
 1128 1433 1355 1334 1029 1124 1343 1375 1289 1231 1337 1323 1236 1437
 1332 1367 1134 1245 1065 1314 1094 1385 1388 1136 1139 1369 1391 1175
 1259 1115 1365 1125 1102 1151 1177 1184 1336 1309 1352 1212 1432 1145
 1405 1284 1408 1108 1402 1380 1038 1445 1140 1069 1123 1254 1317 1430
 1425 1296 1147 1387 1057 1095 1106 1406 1040 1150 1215 1004 1202 1199
 1173 1034 1061 1419 1313 1255 1282 1185 1080 1033 1265 1354 1204 1269
 1152 1035 1454 1283 1143 1345 1160 1209 1153 1444 1247 1103 1384 1431
 1171 1116 1044 1286 1187 1288 1435 1281 1392 1007 1037 1227 1031 1386
 1010 1181 1109 1099 1138 1277 1224 1047 1399 1421 1239 1210 1068 1357
 1170 1148 1101 1237 1192 1011 1279 1307 1180 1036 1017 1213 1089 1280
 1155 1452 1407 1110 1064 1303 1331 1071 1448 1366 1416 1149 1390 1098
 1240 

OneHotEncoder(sparse=False)

In [31]:
ohe.categories_

[array([1001, 1002, 1004, 1005, 1007, 1008, 1009, 1010, 1011, 1012, 1013,
        1014, 1015, 1016, 1017, 1019, 1021, 1022, 1024, 1025, 1026, 1027,
        1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038,
        1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1049, 1050, 1051,
        1052, 1053, 1054, 1056, 1057, 1058, 1060, 1061, 1062, 1063, 1064,
        1065, 1066, 1067, 1068, 1069, 1071, 1072, 1073, 1074, 1075, 1076,
        1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1087, 1088,
        1089, 1090, 1092, 1093, 1094, 1095, 1096, 1098, 1099, 1100, 1101,
        1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112,
        1113, 1114, 1115, 1116, 1117, 1118, 1121, 1123, 1124, 1125, 1127,
        1128, 1129, 1130, 1133, 1134, 1135, 1136, 1138, 1139, 1140, 1141,
        1142, 1143, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153,
        1155, 1156, 1157, 1158, 1159, 1160, 1162, 1163, 1165, 1166, 1167,
        1169, 1170, 1171, 1173, 1175, 

In [32]:
code_commune_encoded = ohe.transform(data_v1[['code_commune']]) # Encode

In [33]:
code_commune_encoded.T.shape

(582, 10000)

In [34]:
dicts_commune = {}
keys = list(ohe.categories_[0])
values = code_commune_encoded.T.astype(int)
for i,j in enumerate(keys):
    dicts_commune[j] = values[i,:]

result = pd.DataFrame.from_dict(dicts_commune)

print(result.shape)
result.head()

(10000, 582)


,1001,1002,1004,1005,1007,1008,1009,1010,1011,1012,...,2790,2791,2803,2807,2819,2820,2821,2824,2827,2834
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
data_v1.shape

(10000, 52)

In [36]:
data_v1 = data_v1.reset_index(drop=True)

In [37]:
#Concat data_v1 and result dataframe
data_v2 = pd.concat([data_v1, result], axis = 1)
data_v2.shape

(10000, 634)

In [38]:
data_v2 = data_v2.drop(columns= 'code_commune')
data_v2.shape

(10000, 633)

In [39]:
data_v2.head(2)

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,code_departement,...,2790,2791,2803,2807,2819,2820,2821,2824,2827,2834
0,2016-1,2016-01-08,Vente,40000,,RUE TONY REVILLON,0560,1750,Saint-Laurent-sur-Saône,1,...,0,0,0,0,0,0,0,0,0,0
1,2016-4,2016-01-08,Vente,90000,,PELUS,B203,1560,Saint-Jean-sur-Reyssouze,1,...,0,0,0,0,0,0,0,0,0,0


## code_nature_culture_speciale

In [40]:
data_v2['code_nature_culture_speciale'].unique()

array(['', 'JARD', 'POTAG', 'PARC', 'MARAI', 'FER', 'PATUR', 'FRICH'],
      dtype=object)

In [41]:
data_v2['code_nature_culture_speciale'].replace("", "NoCodeNatureSpecial", inplace=True) #Replace NaN by "NoCodeNatureSpecial"

In [42]:
print(data_v2['code_nature_culture_speciale'].unique())  # Check unique values 

ohe = OneHotEncoder(sparse = False) # Instanciate encoder

ohe.fit(data_v2[['code_nature_culture_speciale']]) # Fit encoder  ---> OneHotEncoder(sparse=False)

['NoCodeNatureSpecial' 'JARD' 'POTAG' 'PARC' 'MARAI' 'FER' 'PATUR' 'FRICH']


OneHotEncoder(sparse=False)

In [43]:
ohe.categories_

[array(['FER', 'FRICH', 'JARD', 'MARAI', 'NoCodeNatureSpecial', 'PARC',
        'PATUR', 'POTAG'], dtype=object)]

In [44]:
code_nature_culture_special_encoded = ohe.transform(data_v2[['code_nature_culture_speciale']]) # Encode

In [45]:
code_nature_culture_special_encoded.T.shape

(8, 10000)

In [46]:
dicts_nature_special = {}
keys = list(ohe.categories_[0])
values = code_nature_culture_special_encoded.T.astype(int)
for i,j in enumerate(keys):
    dicts_nature_special[j] = values[i,:]

result = pd.DataFrame.from_dict(dicts_nature_special)

print(result.shape)
result.head()

(10000, 8)


,FER,FRICH,JARD,MARAI,NoCodeNatureSpecial,PARC,PATUR,POTAG
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0


In [47]:
data_v2 = data_v2.reset_index(drop=True)

In [48]:
#Concat data_v and result dataframe
data_v3 = pd.concat([data_v2, result], axis = 1)
data_v3.shape

(10000, 641)

In [49]:
data_v3 = data_v3.drop(columns= ['NoCodeNatureSpecial', 'code_nature_culture_speciale'])
data_v3.shape

(10000, 639)

In [50]:
data_v3.head(2)

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,code_departement,...,2824,2827,2834,FER,FRICH,JARD,MARAI,PARC,PATUR,POTAG
0,2016-1,2016-01-08,Vente,40000,,RUE TONY REVILLON,0560,1750,Saint-Laurent-sur-Saône,1,...,0,0,0,0,0,0,0,0,0,0
1,2016-4,2016-01-08,Vente,90000,,PELUS,B203,1560,Saint-Jean-sur-Reyssouze,1,...,0,0,0,0,0,0,0,0,0,0


# Function encoding_categ_column

In [51]:
def encoding_categ_column(df, col_name):
    
    L = list(df[col_name].unique())
    if '' in L:
        df[col_name].replace("", "NoValue", inplace=True) #Replace NaN by "NoCodeNature"
    
    ohe = OneHotEncoder(sparse = False) # Instanciate encoder
    ohe.fit(df[[col_name]]) # Fit encoder  ---> OneHotEncoder(sparse=False)
    
    col_encoded = ohe.transform(df[[col_name]]) # Encode

    dicts_col = {}
    keys = list(ohe.categories_[0])
    values = col_encoded.T.astype(int)
    
    for i,j in enumerate(keys):
        dicts_col[j] = values[i,:]

    result = pd.DataFrame.from_dict(dicts_col)
    
    df = df.reset_index(drop=True)
    
    #Concat df and result dataframes
    data_res = pd.concat([df, result], axis = 1)
    
    if 'NoValue' in list(data_res.columns):
        data_res = data_res.drop(columns= ['NoValue',col_name] )
    else:
        data_res = data_res.drop(columns= col_name)
        
    return data_res   

## Testing function

In [52]:
#Template for getting some data
df2 = get_data("""select * from data_working_update LIMIT 10000""", 1000)

Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows
Got dataframe w/1000 rows


In [53]:
df2.shape

(10000, 37)

In [54]:
data_t = encoding_categ_column(df = df2, col_name = 'code_commune')
data_t.shape

(10000, 618)

In [55]:
L1 = len(df2['code_commune'].unique()) + df2.shape[1] - 1
L2 = data_t.shape[1]
assert(L1 == L2)

In [56]:
data_t2 = encoding_categ_column(df = data_t, col_name = 'code_nature_culture')
data_t2.shape

(10000, 631)

In [57]:
L1 = len(data_t['code_nature_culture'].unique()) + data_t.shape[1] - 2 # 2 because we dropped NoCodeNature column
                                                                    # and encoded col
L2 = data_t2.shape[1]
print(L1, L2)
assert(L1 == L2)

631 631


In [58]:
data_t3 = encoding_categ_column(df = data_t2, col_name = 'type_local')
data_t3.shape

(10000, 633)

In [59]:
L1 = len(data_t2['type_local'].unique()) + data_t2.shape[1] - 1
L2 = data_t3.shape[1]
assert(L1 == L2)

In [60]:
data_t4 = encoding_categ_column(df = data_t3, col_name = 'code_nature_culture_speciale')
data_t4.shape

(10000, 639)

In [61]:
L1 = len(data_t3['code_nature_culture_speciale'].unique()) + data_t3.shape[1] - 2 # 2 because we dropped NoCodeNature column
                                                                    # and encoded col
L2 = data_t4.shape[1]
assert(L1 == L2)

# Function : convert_date

In [62]:
def convert_date(string, separator):
    a = string.split(separator)
    year = a[0]
    month = a[1]
    day = a[2]
    return year, month, day

Year, Month, Day = convert_date(string = '2020-01-09', separator = '-')

In [63]:
Year

'2020'

In [64]:
Month

'01'

In [65]:
Day

'09'